In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as wg
import IPython.display
import pandas as pd
import numpy as np
from scipy import signal
from scipy.signal import get_window
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore")

def plot_windows(window):
    co2 = pd.read_csv("../data/co2.csv") # Tring fft on co2 data
    co2["date"] = pd.to_datetime(co2["date"], format = '%Y-%m-%d') # Convert date column to datetime
    dt = 1 / 12 # yearly freq
    t = np.array(co2["date"]) 
    f = np.array(co2["CO2"])
    n = len(t)

    fhat = np.fft.fft(f, n) # Compute FFT
    PSD = fhat * np.conj(fhat) / n # Power Spectrum
    if window == "tukey":
        window = signal.tukey(len(f))
    else:
        window = get_window(window, len(f))

    PSD = PSD*window # Multiply PSD by window (multipliction in frequency domain is convultion in time domain)
    freq = (1 / (dt*n)) * np.arange(n) # Find frequency
    L = np.arange(1, np.floor(n/2), dtype = 'int')  # Only want first half of the frequency domain
    peaks = int(np.quantile(PSD, .985)) # Use the PSD to filter out noise
    indices = PSD < peaks # Filter out freqs with low power

    fhat_clean = indices * fhat # Zero out small Fourier coefs
    ffilt = np.fft.ifft(fhat_clean) # Inverse FFT for filtered time signal

    indices_seasonal = PSD >= peaks # Filter out freqs with high power
    fhat_seasonal = indices_seasonal * fhat # Zero out large Fourier coefs
    ffilt_seasonal = np.fft.ifft(fhat_seasonal) # Inverse FFT for filtered time signal


    fig, axes = plt.subplots(5, 1)

    plt.sca(axes[0])
    plt.plot(sorted(freq[L]),PSD[L], color = 'c')
    plt.xlim(freq[L[0]], freq[L[-1]])
    plt.xlabel("Freq (1/yr)")
    plt.ylabel("FFT")

    plt.sca(axes[1])
    plt.plot(t, f, color = 'c')
    plt.xlim(t[0], t[-1])
    plt.xlabel("Year")
    plt.ylabel("CO2 Emissions")


    plt.sca(axes[2])
    plt.plot(t, ffilt, color = 'k')
    plt.xlim(t[0], t[-1])

    plt.sca(axes[3])
    plt.plot(t, ffilt_seasonal, color = "red")
    plt.xlim(t[0], t[-1])



    # Find the residuals 
    residuals = co2["CO2"] - ffilt_seasonal - ffilt
    plt.sca(axes[4])
    plt.scatter(t, residuals)
    plt.xlim(t[0], t[-1])

    plt.show()

    # Initial Statistical Summary
    print("The rmse", math.sqrt(mean_squared_error(co2["CO2"], (ffilt_seasonal + ffilt).real)))
    print("The dot product of STF seasonal and trend components is ", np.round(ffilt_seasonal.dot(ffilt)))

In [12]:
from scipy.signal import get_window
window_slider = wg.SelectionSlider(
    options = ["boxcar", "blackman", "hanning", "tukey", "hann", "parzen"],
    value = "boxcar",
    description = "window",
    disabled = False,
    continuous_update = True,
    orientation = "horizontal",
    readout = True
)

wg.interact(plot_windows, window = window_slider)

interactive(children=(SelectionSlider(description='window', options=('boxcar', 'blackman', 'hanning', 'tukey',…

<function __main__.plot_windows(window)>